### SWIMPLACES 

In [8]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import json
import unicodedata

In [59]:
#how to access another than the first page
response = requests.get('https://www.mapotic.com/api/v1/maps/2941/public-pois/?page=1&page_size=10&ordering=created&page=4')
print(response)
#response.json()

<Response [200]>


In [135]:
class Swim():
    '''
    This Class will be used to download all data and merge them into one DataFrame.
    '''
    def __init__(self):
#         raw = self.downloadPages()
#         df = self.getData()
#         at = self.getAtt()
        pass
    def downloadPages(self):
        """Download data from API and save to a json file."""
        
        resp=[]
        b=[1,2,3,4] #přidat automatické zastavení po poslední stránce (místo počítání do 4)
        for i in b:
            response = requests.get('https://www.mapotic.com/api/v1/maps/2941/public-pois/?page=1&page_size=1000&ordering=created&page='+ str(i) )
            resp.append(response.json())
#         if save is True:    
        with open('raw_data.json', 'w') as raw:   # uložit do classy, ale aby se to pokaždé nestahovalo... plus podmínku, kdy se má stáhnout
            json.dump(resp, raw) 
#         return resp
            
    def getData(self):
        """Extract available data from JSON"""
        with open('raw_data.json') as raw_data: 
            preview = json.load(raw_data)
#         preview = self.downloadPages()
        latitude = []    # nějaké kontrolovat délky listů? extrahovat dictionary možná jinak?
        longitude = []
        ids = [] 
        names = [] 
#         types =[]
        ratings = []
        count_ratings = []
        created = []
        for page in preview: 
            for result in page['results']:
                names.append(result['name'])
                ids.append(result['id'])
                latitude.append(result['point']['coordinates'][0])
                longitude.append(result['point']['coordinates'][1])
#                 types.append(result['category']['name']['en'][0])  #z nějakého důvodu nefunguje - NoneType error
                ratings.append(result['rating']['average'])
                count_ratings.append(result['rating']['count'])
                created.append(result['created'])

        df = pd.DataFrame({
            'ID' : ids, 
            'Name' : names, 
#             'type' : types,
            'Average rating' : ratings,
            'Number of ratings' : count_ratings,
            'Created' : created,
            'Longitude' : longitude,
            'Latitude' : latitude})
#         return df 
        return df.loc[:20]  #zde ukáže momentálně 3 výsledky, jen pro kontrolu
    
    def getDescPlaces(self, idn, name):
        """Function to get the source (only the part we are interested in) from each specific website. ID and name (without diacritics) have to be entered."""
        
        name = unicodedata.normalize('NFKD', name).lower()
        new_name = ''
        for c in name:
            if not unicodedata.combining(c):
                new_name += c
                
        odkaz = 'https://www.swimplaces.com/'  + str(idn) + '-'+ new_name.replace(' ', '-') 
        r = requests.get(odkaz)
        r.encoding = 'UTF-8'
        soup = BeautifulSoup(r.text,'lxml').find('meta', {'name' : 'description'})['content'].split('|')
        return soup
    
    def getAtt(self):
        """Download attributes for all the places."""
        value = None
        def find_between(string, start, end):  #pomocná na hledání ve stringu
            return (string.split(start))[1].split(end)[0]
        
        nazev =[]
        att = []
        for i, n in self.getData().iterrows(): 
            nazev.append(self.getDescPlaces(n['ID'], n['Name'])[0])
            att.append(self.getDescPlaces(n['ID'], n['Name'])[1])  # tady už to někdy hodilo chybu, ze to nenašlo...
        ids_att = self.getData()['ID'].values #přidat ID, pro pozdější join
        
        desc = []
        refresh = []
        diving = []
        entrances = []
        access = []
        nudists = []
        for ii in att:  
            if 'Description:' in ii:
                desc.append(find_between(ii, 'Description:', ':').rsplit(' ', 1)[0].rsplit(',', 1)[0].strip()) # najde atribut a vrátí jeho obsah - snad nikde jinde není dvojtečka :D
            else:
                desc.append(value)
            if 'Refreshment' in ii:
                refresh.append(find_between(ii, 'Refreshment:', ':').rsplit(' ', 1)[0].rsplit(',', 1)[0].strip())
            else:
                refresh.append(value) 
            if 'Diving' in ii:
                diving.append(find_between(ii, 'Diving:', ':').rsplit(' ', 1)[0].rsplit(',', 1)[0].strip())
            else:
                diving.append(value)
            if 'Accessibility/parking:' in ii:
                access.append(find_between(ii, 'Accessibility/parking:', ':').rsplit(' ', 1)[0].rsplit(',', 1)[0].strip()) 
            else:
                access.append(value)                                                         
            if 'Entrance' in ii:
                entrances.append(find_between(ii, 'Entrance:', ':').rsplit(' ', 1)[0].rsplit(',', 1)[0].strip()) 
            else:
                entrances.append(value)
            if 'Nudist beach' in ii:
                nudists.append(find_between(ii, 'Nudist beach:', ':').rsplit(' ', 1)[0].rsplit(',', 1)[0].strip())
            else: 
                nudists.append(value)
        attributes = pd.DataFrame({
            'id_a' : ids_att,
            'nazev' :nazev,
            'Description' : desc,
            'Refreshment' : refresh,
            'Diving' : diving,
            'Entrance' : entrances,
            'Accessibility and parking' :access,
            'Nudist beach' : nudists
        })
        return attributes
    def mergeData(self):
        ''' Merge those 2 data sets, cleaning will follow.'''
        output = self.getData().merge(self.getAtt(), left_on = 'ID', right_on = 'id_a')
#         output = output['ids', 'names'] # , 'avg_rating', 'count_ratings', 'created', 'logitude', 'latitude', 'description', 'refreshment', 'diving', 'entrance', 'accessibility_parking', 'nudist_beach']
        return output

In [136]:
swim = Swim()
# swim.downloadPages() # update soup - change 
swim.mergeData()

,ID,Name,Average rating,Number of ratings,Created,Longitude,Latitude,id_a,nazev,Description,Refreshment,Diving,Entrance,Accessibility and parking,Nudist beach
0,202835,Costa Brava,NaN,0,2010-08-21T11:30:00Z,41.760320,2.983775,202835,Costa Brava,Nadherné písčité pláže a relativně málo turistů,No restaurant,Not suitable for diving,No entrance fee,None,Not suitable for nudists
1,202880,Nádrž Dolní Žleb,2.000000,1,2011-06-28T22:21:32Z,49.751182,17.307842,202880,Nádrž Dolní Žleb,V létě hojně využívaná rekreanty,Restaurant on site,Not suitable for diving,No entrance fee,None,Not suitable for nudists
2,202902,Kralupy nad Vltavou,5.000000,3,2011-06-29T12:34:14Z,50.256161,14.307153,202902,Kralupy nad Vltavou,Krásné letní koupaliště na okraji města. Nabíz...,Restaurant on site,Not suitable for diving,Entrance fee,Very close,Not suitable for nudists
3,202917,Koupaliště Flošna,4.800000,5,2011-06-29T14:16:31Z,50.205154,15.841977,202917,Koupaliště Flošna,Koupaliště Flošna Hradec Králové,Restaurant on site,Not suitable for diving,Entrance fee,Very close,Suitable for nudists
4,202918,Městské lázně,4.500000,2,2011-06-29T14:17:45Z,50.214217,15.828759,202918,Městské lázně,Městské lázně - aquacentrum,Restaurant on site,Not suitable for diving,Entrance fee,None,Not suitable for nudists
5,202919,Plavecký bazén,5.000000,2,2011-06-29T14:18:19Z,50.214272,15.830562,202919,Plavecký bazén,Plavecký bazén 50m,Restaurant on site,Not suitable for diving,Entrance fee,None,Not suitable for nudists
6,202921,Koupaliště Divoká Šárka,4.363636,11,2011-06-29T14:21:52Z,50.099060,14.318681,202921,Koupaliště Divoká Šárka,Koupaliště U Veselíka,Restaurant on site,Not suitable for diving,Entrance fee,Very close,Not suitable for nudists
7,202924,Aquadream,3.416667,12,2011-06-29T14:30:04Z,50.026646,14.367819,202924,Aquadream,"aquapark a koupaliště Barrandov, k dispozici j...",Restaurant on site,Not suitable for diving,Entrance fee,Very close,Not suitable for nudists
8,202925,Letňany Lagoon,4.400000,5,2011-06-29T14:32:25Z,50.134352,14.504848,202925,Letňany Lagoon,aquacentrum Letňany Lagoon,Restaurant on site,Not suitable for diving,Entrance fee,Very close,Not suitable for nudists
9,202935,Kemp Dívčí kámen,4.000000,1,2011-06-29T15:56:17Z,48.892115,14.361025,202935,Kemp Dívčí kámen,"krásně prosluněná louka, osvěžující voda z řek...",Restaurant on site,Not suitable for diving,No entrance fee,None,Not suitable for nudists


__Next steps:__
- improve the Class
- Clean dataframe & check validity
- Analysis of data:
    - Descriptive stats
    - Visualization